In [18]:
import numpy as np
import random
import math

# Define the grid size and obstacles
grid_size = 10  # 10x10 grid
obstacles = [(2, 2), (3, 3), (4, 4)]  # Obstacles (represented as points)
start_point = (0, 0)  # Start point
goal_point = (9, 9)   # Goal point

# Function to calculate Euclidean distance between two points
def euclidean_distance(p1, p2):
    return np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)

# Function to check if a point is within a certain radius of an obstacle
def is_collision(point, threshold=1.0):
    # Check if the point is within the threshold distance from any obstacle
    for obstacle in obstacles:
        if euclidean_distance(point, obstacle) < threshold:
            return True  # Return True if the point is near an obstacle
    return False  # Return False if the point is not near any obstacle

# Objective function to evaluate path
def path_planning(path):
    # Check if path collides with obstacles
    for point in path:
        if is_collision(point):
            return float('inf')  # Return a large penalty for collisions

    # Calculate total path length
    path_length = 0
    for i in range(len(path) - 1):
        path_length += euclidean_distance(path[i], path[i+1])

    # Return path length as fitness (minimizing path length)
    return path_length

# Lévy flight for generating new paths
def levy_flight(Lambda, dim, lb, ub):
    sigma_u = (math.gamma(1 + Lambda) * math.sin(math.pi * Lambda / 2) /
               (math.gamma((1 + Lambda) / 2) * Lambda * 2**((Lambda - 1) / 2)))**(1 / Lambda)
    u = np.random.normal(0, sigma_u, dim)
    v = np.random.normal(0, 1, dim)
    step = u / np.abs(v)**(1 / Lambda)

    # Return step for each dimension (to modify each waypoint's coordinate)
    return step

# Cuckoo Search Algorithm for Path Planning
def cuckoo_search(func, dim, num_nests=20, max_iter=1000, pa=0.25, lb=0, ub=grid_size-1):
    # Initialize nests (paths) randomly
    nests = [np.random.uniform(lb, ub, (dim, 2)).tolist() for _ in range(num_nests)]  # Random continuous paths
    fitness = np.array([func(nest) for nest in nests])

    # Track the best solution
    best_nest = nests[np.argmin(fitness)]
    best_fitness = min(fitness)

    # Iterate for the specified number of iterations
    for iteration in range(max_iter):
        # Generate new solutions using Lévy flights
        for i in range(num_nests):
            step = levy_flight(1.5, dim, lb, ub)  # Lévy flight with Lambda=1.5
            new_solution = []

            # Apply step to each coordinate in the path
            for j in range(dim):
                new_point = nests[i][j] + step[j]
                # Clip the new point to ensure it stays within bounds
                new_point = np.clip(new_point, lb, ub)
                new_solution.append(new_point)

            new_fitness = func(new_solution)

            # If new solution is better, replace the old nest
            if new_fitness < fitness[i]:
                nests[i] = new_solution
                fitness[i] = new_fitness

        # Abandon worst nests and generate new ones
        for i in range(num_nests):
            if random.random() < pa:  # Discovery probability
                nests[i] = np.random.uniform(lb, ub, (dim, 2)).tolist()  # Reinitialize nest
                fitness[i] = func(nests[i])

        # Update the best solution found so far
        best_idx = np.argmin(fitness)
        if fitness[best_idx] < best_fitness:
            best_nest = nests[best_idx]
            best_fitness = fitness[best_idx]

    return best_nest, best_fitness

# Main code to run the algorithm
if __name__ == "__main__":
    # Set the dimensions for the path (a sequence of 2D points)
    dim = 10  # Number of points in the path (i.e., length of the path)

    # Run the Cuckoo Search for path planning
    best_path, best_fitness = cuckoo_search(path_planning, dim=dim, num_nests=25, max_iter=1000, pa=0.25)

    # Output the best path and fitness in a standardized format
    print("\nBest Path Found:")
    print("Start Point: ", start_point)
    print("Goal Point: ", goal_point)
    print("\nPath Coordinates (Avoiding Obstacles):")
    for idx, point in enumerate(best_path):
        collision = "Avoided Obstacle" if is_collision(point) else "No Collision"
        print(f"Waypoint {idx + 1}: {tuple(point)} ({collision})")

    print(f"\nTotal Path Length: {best_fitness:.4f}")



Best Path Found:
Start Point:  (0, 0)
Goal Point:  (9, 9)

Path Coordinates (Avoiding Obstacles):
Waypoint 1: (np.float64(7.0959648864859215), np.float64(7.586748309596009)) (No Collision)
Waypoint 2: (np.float64(7.936383693278471), np.float64(9.0)) (No Collision)
Waypoint 3: (np.float64(6.40192537746959), np.float64(7.323116283222218)) (No Collision)
Waypoint 4: (np.float64(7.288434333166902), np.float64(6.051875106554889)) (No Collision)
Waypoint 5: (np.float64(5.657085585560215), np.float64(6.928295602613859)) (No Collision)
Waypoint 6: (np.float64(6.4644649118840904), np.float64(6.317538318326444)) (No Collision)
Waypoint 7: (np.float64(4.023208896729978), np.float64(7.0784675757352)) (No Collision)
Waypoint 8: (np.float64(9.0), np.float64(9.0)) (No Collision)
Waypoint 9: (np.float64(8.612120877006127), np.float64(8.612120877006127)) (No Collision)
Waypoint 10: (np.float64(8.945749397073646), np.float64(9.0)) (No Collision)

Total Path Length: 17.2834
